In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from ipywidgets import interact, widgets
from gan import get_random_generator_input, get_generator_sample
from gan import load_params, model_dir
from human_pose_util.register import skeleton_register, dataset_register
from human_pose_util.register import register_defaults
from human_pose_util.skeleton import vis3d

In [2]:
model_name = 'big'

In [3]:
register_defaults()
params = load_params(model_name)
skeleton = skeleton_register[
    dataset_register[params['dataset']]['train'].attrs['skeleton_id']]
n_z = params['n_z']

In [4]:
z_vals = np.zeros((n_z,), dtype=np.float32)

In [5]:
graph = tf.Graph()
with graph.as_default():
    z = tf.placeholder(shape=(1, n_z,), dtype=tf.float32, name='z')
    sample = get_generator_sample(z, params, reuse=False)
    generator_vars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES)

In [6]:
sess = tf.InteractiveSession(graph=graph)
saver = tf.train.Saver(var_list=generator_vars)
saver.restore(sess, tf.train.latest_checkpoint(model_dir(model_name)))

INFO:tensorflow:Restoring parameters from /home/jackd/Development/python/adversarially_parameterized_optimization/models/big/model.ckpt-9868002


In [7]:
@interact(i=range(n_z), z_val=(-3.0, 3.0))
def generate(i=0, z_val=0.):
    z_vals[i] = z_val
    pose = sess.run(sample, feed_dict={z: np.expand_dims(z_vals, axis=0)})[0]
    ax = vis3d(skeleton, pose)
    r = 0.6
    ax.set_zlim3d(0, 2*r)
    ax.set_xlim3d(-r, r)
    ax.set_ylim3d(-r, r)
    plt.show()

interactive(children=(Dropdown(description=u'i', options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31), value=0), FloatSlider(value=0.0, description=u'z_val', max=3.0, min=-3.0), Output()), _dom_classes=('widget-interact',))

In [8]:
sess.close()